In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]
LEARNING_RATE = 1e-3
NUM_EPOCHS = 30

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)), 
    transforms.RandomHorizontalFlip(),                    
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),                         
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),                               
    transforms.Normalize(MEAN, STD)                    
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

In [ ]:
train_dataset_path = "../../../dataset/train"
val_dataset_path = "../../../dataset/val"

train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_dataset_path, transform=val_transforms)

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS,
    pin_memory=True)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"Classes detected: {train_dataset.classes}")


In [ ]:
model = models.densenet201(weights=models.DenseNet201_Weights.IMAGENET1K_V1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, correct,total = 0.0, 0, 0
    
    for images, labels, in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    return running_loss/total, correct/total
    

In [ ]:
def validate(model, loader, criterion, device):
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
    return val_loss/total, correct/total

In [ ]:
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}: "
          f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"Val loss {val_loss:.4f}, acc {train_acc:.4f}"
          )
    
    if(val_loss < best_val_loss):
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_densenet201.pth")

Total batches: 290
Epoch 1/30: Train loss 0.9940, acc 0.6956 | Val   loss 0.4941, acc 0.8717
Epoch 2/30: Train loss 0.5772, acc 0.8145 | Val   loss 0.4204, acc 0.8657
Epoch 3/30: Train loss 0.5042, acc 0.8329 | Val   loss 0.3051, acc 0.9101
Epoch 4/30: Train loss 0.4521, acc 0.8489 | Val   loss 0.2990, acc 0.9121
Epoch 5/30: Train loss 0.4311, acc 0.8504 | Val   loss 0.2829, acc 0.9141
Epoch 6/30: Train loss 0.4160, acc 0.8545 | Val   loss 0.2590, acc 0.9303
Epoch 7/30: Train loss 0.4340, acc 0.8459 | Val   loss 0.2470, acc 0.9253
Epoch 8/30: Train loss 0.4345, acc 0.8454 | Val   loss 0.2904, acc 0.8990
Epoch 9/30: Train loss 0.4257, acc 0.8519 | Val   loss 0.2590, acc 0.9131
Epoch 10/30: Train loss 0.4145, acc 0.8517 | Val   loss 0.2287, acc 0.9343
Epoch 11/30: Train loss 0.4035, acc 0.8573 | Val   loss 0.2307, acc 0.9202
Epoch 12/30: Train loss 0.4304, acc 0.8465 | Val   loss 0.2216, acc 0.9182
Epoch 13/30: Train loss 0.3932, acc 0.8578 | Val   loss 0.2636, acc 0.9051
Epoch 14/30: Tr